In [ ]:
#importing some useful packages
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
%matplotlib inline


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def polyfit(img, x, y, color=[255, 0, 0], thickness=10):
    """
    Extrapolate to the top and bottom of the lane, ranging from y=imageSize to y=320.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    fit = np.polyfit( x, y, 1)

    imshape = img.shape
    ymin = imshape[0]
    ymax = 320
    xmin = int((ymin - fit[1])/fit[0])
    xmax = int((ymax - fit[1])/fit[0])
    cv2.line(img, (xmin, ymin), (xmax, ymax), color, thickness)
    

def draw_lines(img, lines):
    """    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then we discard line whose absolute slope is 
    less htan a threshold, followed by applying polyFit of order 1 to them.
    """

    left_x=[]
    right_x=[]
    left_y = []
    right_y = []
        
    for line in lines:
        for x1,y1,x2,y2 in line:
            #print("1", x1, x2, y1, y2)
            if((x2-x1)!=0):
                slope = float((y2-y1)/(x2-x1))
            if abs(slope) < 0.1:
                continue
            if (slope > 0):
                left_x.append(x1)
                left_x.append(x2)
                left_y.append(y1)
                left_y.append(y2)
            else:
                right_x.append(x1)
                right_x.append(x2)
                right_y.append(y1)
                right_y.append(y2)
    
    polyfit(img, left_x, left_y)
    polyfit(img, right_x, right_y)
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



def process_image(image):
    # Read in the image
    imshape = image.shape
    gaussian_kernal_size = 5
    canny_low_threshold = 50
    canny_high_threshold = 150
    roi_vertices = np.array([[(150,imshape[0]),(440, 320), (500, 320), (imshape[1]-10,imshape[0])]], dtype=np.int32)
    # Define the Hough transform parameters
    hough_rho = 2 # distance resolution in pixels of the Hough grid
    hough_theta = np.pi/180 # angular resolution in radians of the Hough grid
    hough_threshold = 50     # minimum number of votes (intersections in Hough grid cell)
    hough_min_line_length = 5 #minimum number of pixels making up a line
    hough_max_line_gap = 1    # maximum gap in pixels between connectable line segments


    # Grayscale the image
    gray = grayscale(image)
    # Apply gaussian blur on the image
    gBlur = gaussian_blur(gray, gaussian_kernal_size)
    # Apply Canny Edge Detector and get the edges
    cEdges = canny(gBlur, canny_low_threshold, canny_high_threshold)
    # Defining a four sided polygon to mask
    masked_cEdges = region_of_interest(cEdges, roi_vertices)
    # Run Hough on polygon masked image. Iterate over the output "lines" and draw lines on a blank image
    hLines = hough_lines(masked_cEdges, hough_rho, hough_theta, hough_threshold, hough_min_line_length, hough_max_line_gap)
    # Draw the lines on the edge image
    line_image = weighted_img(hLines, image)
    return line_image


yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)
